In [85]:
import numpy as np

# Constants and Parameters
H_m = 1e-96  # Consumption factor of electricity
y_nt = 1e-6  # Weighted factor of local time cost
y_ne = 1e-12  # Weighted factor of local energy consumption
e_m = 1e-15  # Energy required to calculate a single bit of task data for MEC server
noise_power_spectral_density = 1e-9  # in Watts/Hz
channel_gain = 2e-10  
bandwidth = 1e9  # Corrected: 1 GHz bandwidth
transmission_power = 0.5  # in Watts
device_computing_capacity = np.array([0.5, 1])  # in GHz, now as a NumPy array
mec_computing_capacity = 4  # in GHz
task_data = np.array([5, 10, 15, 20, 25, 30, 35, 40, 45]) * 1e6  # in bits, converted from Mbit

# DAG representation with all tasks accounted for
dag_dependencies = {
    0: [1],
    1: [2, 3],
    2: [],  
    3: [4],
    4: [],
}

def topological_sort(dag_dependencies):
    from collections import deque

    all_nodes = set(dag_dependencies.keys()) | set(v for values in dag_dependencies.values() for v in values)
    in_degree = {node: 0 for node in all_nodes}
    for values in dag_dependencies.values():
        for v in values:
            in_degree[v] += 1

    queue = deque([node for node in in_degree if in_degree[node] == 0])
    sorted_order = []

    while queue:
        node = queue.popleft()
        sorted_order.append(node)
        for adjacent in dag_dependencies.get(node, []):
            in_degree[adjacent] -= 1
            if in_degree[adjacent] == 0:
                queue.append(adjacent)

    if len(sorted_order) != len(all_nodes):
        raise ValueError("The DAG has a cycle, and a topological sort is not possible.")

    return sorted_order

class QPSO:
    def __init__(self, num_devices, num_tasks, max_iter, task_data, dag_dependencies, device_computing_capacity, mec_computing_capacity):
        self.num_devices = num_devices
        self.num_tasks = num_tasks
        self.max_iter = max_iter
        self.task_data = task_data
        self.dag_dependencies = dag_dependencies
        self.device_computing_capacity = device_computing_capacity
        self.mec_computing_capacity = mec_computing_capacity
        self.X = np.random.randint(2, size=(num_devices, num_tasks, max_iter))  # Initial binary decisions for each device and task
        self.P = np.zeros((num_devices, num_tasks, max_iter), dtype=int)  # Personal best decisions
        self.P_fitness = np.full((num_devices, 1), np.inf)  # Personal best fitness values for each device
        self.g = np.zeros((num_tasks,), dtype=int)  # Global best decision, assuming a single best strategy across devices for simplicity
        self.g_fitness = np.inf  # Global best fitness value

    def fitness_function(self, decision_vector):
        completion_times = np.zeros(len(self.task_data))
        total_energy = 0
        sorted_tasks = topological_sort(self.dag_dependencies)
        device_capacity_mean = np.mean(self.device_computing_capacity) * 1e9  # Using mean capacity for deterministic calculation

        for task in sorted_tasks:
            task_size = self.task_data[task]
            predecessors = [k for k, v in self.dag_dependencies.items() if task in v]
            start_time = max(completion_times[pre] for pre in predecessors) if predecessors else 0

            if (decision_vector[:, task] == 0).all():  # Local computation
                execution_time = task_size / device_capacity_mean
                energy_cost = H_m * device_capacity_mean ** 2 * task_size
            else:  # Edge computation
                transmission_time = task_size / (bandwidth * np.log2(1 + (transmission_power * channel_gain) / (noise_power_spectral_density * bandwidth)))
                execution_time = task_size / (self.mec_computing_capacity * 1e9)
                energy_cost = transmission_power * transmission_time + e_m * task_size
                execution_time += transmission_time

            completion_times[task] = start_time + execution_time
            total_energy += energy_cost

        total_time = max(completion_times)
        total_cost = y_nt * total_time + y_ne * total_energy
        
        return total_cost

    def run(self):
        for t in range(1, self.max_iter):
            for i in range(self.num_devices):
                for j in range(self.num_tasks):  # Adjusted loop over tasks
                    phi = np.random.uniform(0, 1)
                    self.X[i, j, t] = (phi * self.P[i, j, t-1] + (1 - phi) * self.g[j]) % 2  # Ensuring binary decision


                current_fitness = self.fitness_function(self.X[:, t])
                if current_fitness < self.P_fitness[i]:
                    self.P[i, t] = self.X[i, t]
                    self.P_fitness[i] = current_fitness

                if current_fitness < self.g_fitness:
                    self.g = self.X[:, t].copy()  # Ensure a proper copy is made
                    self.g_fitness = current_fitness

        return self.g, self.g_fitness

num_tasks = len(task_data)
max_iter = 100
num_devices = 2  # Assuming there are two devices, adjust as needed

qpso = QPSO(num_devices, num_tasks, max_iter, task_data, dag_dependencies, device_computing_capacity, mec_computing_capacity)
best_strategy, best_fitness = qpso.run()

print("Best offloading strategy:", best_strategy)
print("Best fitness value:", best_fitness)


ValueError: setting an array element with a sequence.

In [86]:
import numpy as np

# Constants and Parameters
H_m = 1e-96  # Consumption factor of electricity
y_nt = 1e-6  # Weighted factor of local time cost
y_ne = 1e-12  # Weighted factor of local energy consumption
e_m = 1e-15  # Energy required to calculate a single bit of task data for MEC server
noise_power_spectral_density = 1e-9  # in Watts/Hz
channel_gain = 2e-10  
bandwidth = 1e9  # Corrected: 1 GHz bandwidth
transmission_power = 0.5  # in Watts
device_computing_capacity = np.array([0.5, 1])  # in GHz, now as a NumPy array
mec_computing_capacity = 4  # in GHz
task_data = np.array([5, 10, 15, 20, 25, 30, 35, 40, 45]) * 1e6  # in bits, converted from Mbit

# DAG representation with all tasks accounted for
dag_dependencies = {
    0: [1],
    1: [2, 3],
    2: [],  
    3: [4],
    4: [],
}

def topological_sort(dag_dependencies):
    from collections import deque

    all_nodes = set(dag_dependencies.keys()) | set(v for values in dag_dependencies.values() for v in values)
    in_degree = {node: 0 for node in all_nodes}
    for values in dag_dependencies.values():
        for v in values:
            in_degree[v] += 1

    queue = deque([node for node in in_degree if in_degree[node] == 0])
    sorted_order = []

    while queue:
        node = queue.popleft()
        sorted_order.append(node)
        for adjacent in dag_dependencies.get(node, []):
            in_degree[adjacent] -= 1
            if in_degree[adjacent] == 0:
                queue.append(adjacent)

    if len(sorted_order) != len(all_nodes):
        raise ValueError("The DAG has a cycle, and a topological sort is not possible.")

    return sorted_order

class QPSO:
    def __init__(self, num_devices, num_tasks, max_iter, task_data, dag_dependencies, device_computing_capacity, mec_computing_capacity):
        self.num_devices = num_devices
        self.num_tasks = num_tasks
        self.max_iter = max_iter
        self.task_data = task_data
        self.dag_dependencies = dag_dependencies
        self.device_computing_capacity = device_computing_capacity
        self.mec_computing_capacity = mec_computing_capacity
        self.X = np.random.randint(2, size=(num_devices, num_tasks, max_iter))  # Initial binary decisions for each device and task
        self.P = self.X.copy()  # Personal best decisions
        self.P_fitness = np.full((num_devices, max_iter), np.inf)  # Personal best fitness values for each device
        self.g = np.zeros((num_tasks,), dtype=int)  # Global best decision, assuming a single best strategy across devices for simplicity
        self.g_fitness = np.inf  # Global best fitness value

    def fitness_function(self, decision_vector):
        completion_times = np.zeros(len(self.task_data))
        total_energy = 0
        sorted_tasks = topological_sort(self.dag_dependencies)
        device_capacity_mean = np.mean(self.device_computing_capacity) * 1e9  # Using mean capacity for deterministic calculation

        for task in sorted_tasks:
            task_size = self.task_data[task]
            predecessors = [k for k, v in self.dag_dependencies.items() if task in v]
            start_time = max(completion_times[pre] for pre in predecessors) if predecessors else 0

            if (decision_vector[:, task] == 0).all():  # Local computation
                execution_time = task_size / device_capacity_mean
                energy_cost = H_m * device_capacity_mean ** 2 * task_size
            else:  # Edge computation
                transmission_time = task_size / (bandwidth * np.log2(1 + (transmission_power * channel_gain) / (noise_power_spectral_density * bandwidth)))
                execution_time = task_size / (self.mec_computing_capacity * 1e9)
                energy_cost = transmission_power * transmission_time + e_m * task_size
                execution_time += transmission_time

            completion_times[task] = start_time + execution_time
            total_energy += energy_cost

        total_time = max(completion_times)
        total_cost = y_nt * total_time + y_ne * total_energy
        
        return total_cost

    def run(self):
        for t in range(1, self.max_iter):
            for i in range(self.num_devices):
                for j in range(self.num_tasks):  # Adjusted loop over tasks
                    phi = np.random.uniform(0, 1)
                    self.X[i, j, t] = (phi * self.P[i, j, t-1] + (1 - phi) * self.g[j]) % 2  # Ensuring binary decision

                current_fitness = self.fitness_function(self.X[:, :, t])  # Corrected fitness function input
                self.P_fitness[i, t] = current_fitness  # Updated personal best fitness

                if current_fitness < self.g_fitness:
                    self.g = self.X[:, :, t].copy()  # Update global best decision
                    self.g_fitness = current_fitness  # Update global best fitness

        return self.g, self.g_fitness

num_tasks = len(task_data)
max_iter = 100
num_devices = 2  # Assuming there are two devices, adjust as needed

qpso = QPSO(num_devices, num_tasks, max_iter, task_data, dag_dependencies, device_computing_capacity, mec_computing_capacity)
best_strategy, best_fitness = qpso.run()

print("Best offloading strategy:", best_strategy)
print("Best fitness value:", best_fitness)


ValueError: setting an array element with a sequence.